# Type Hinting

In [ ]:
def get_full_name(first_name, last_name:str):
    full_name = first_name.title() + " " + last_name
    return full_name

print(get_full_name("john", "doe"))

Type hinting enables suggesttions. For example:

full_name = frist_name.sth

There will be a list of options

# FastAPI

In [ ]:
from enum import Enum

from fastapi import FastAPI


class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"


app = FastAPI()


@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name is ModelName.alexnet:
        return {"model_name": model_name, "message": "Deep Learning FTW!"}

    if model_name.value == "lenet":
        return {"model_name": model_name, "message": "LeCNN all the images"}

    return {"model_name": model_name, "message": "Have some residuals"}

access /models/alexnet -> get "Deep Learning FTW!"

access /models/lenet -> get "LeCNN all the images"

access /models/resnet -> get "Have some residuals"

## Inheritance

In [1]:
class Animal:
    def speak(self):
        print("Some sound")

class Dog(Animal):
    pass

dog = Dog()
dog.speak()

Some sound


## Enum

In [ ]:
class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"

print(ModelName.alexnet)          # ModelName.alexnet
print(ModelName.alexnet.value)    # alexnet

## Parameters

If more than 1 layer, use

:path

In [ ]:
@app.get("/files/{file_path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

/files/hello.txt                    -> Can access
/files/home/johndoe/myfile/txt      -> Cannot access

In [ ]:
@app.get("/files/{file_path:path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

请求路径	                             匹配结果

/files/hello.txt	                    file_path = "hello.txt"

/files/home/johndoe/myfile.txt	        file_path = "home/johndoe/myfile.txt"

In [ ]:
@app.get("/items/{item_id}")
async def read_item(item_id: str, p: str, q: str | None = None):
    if q:
        return {"item_id": item_id, "q": q, "p": p}
    return {"item_id": item_id, "p": p}

item_id is a **path parameter**, since it's in {}: 

/items/apple

/items/123

/items/xyz

q is a **query parameter**:

/items/apple?q=hello&p=hola

Here, q is optional and p is not. Optional parameters must come after non-optional ones. 


## Request Body 请求体

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel # Verify input structure


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


app = FastAPI()


@app.post("/items/")
async def create_item(item: Item):
    return item

请求体的作用

1. 传递复杂数据结构

    JSON 可以表示嵌套对象、列表、字典，非常灵活。

    Pydantic 模型 Product 自动验证类型、必填字段、默认值。

2. 清晰的 API 语义

    路径参数 → 哪个资源

    请求体 → 更新后的内容

    查询参数 → 控制或可选参数

3. 安全性与扩展性

    可以加密或签名 JSON

    可以随时扩展字段而不破坏 API

## Extra Verification

### Query Parameters

In [ ]:
from typing import Annotated

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(
    q: Annotated[
        str | None, Query(min_length=3, max_length=50, pattern="^fixedquery$")
    ] = None,
):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

The input can be a list. 

http://localhost:8000/items/?q=foo&q=bar

In [ ]:
from typing import Annotated

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: Annotated[list[str] | None, Query()] = None):
    query_items = {"q": q}
    return query_items

If the query parameter is not a valid python variable, e.g. item-query

Use alias

In [ ]:
@app.get("/items/")
async def read_items(q: Annotated[str | None, Query(alias="item-query")] = None):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

### Path Parameters:

In [ ]:
from typing import Annotated

from fastapi import FastAPI, Path, Query

app = FastAPI()


@app.get("/items/{item_id}")
async def read_items(
    item_id: Annotated[int, Path(title="The ID of the item to get")],
    q: Annotated[str | None, Query(alias="item-query")] = None,
):
    results = {"item_id": item_id}
    if q:
        results.update({"q": q})
    return results

Unlike query parameters, path parameters are always non-optional

In [ ]:
def f(*, b, a): # After * key-word only
    ...
f(a=1, b=2)  # OK，顺序可调
f(1, 2)      # ❌ 错误

In [ ]:
async def read_items(
    *, q: str, item_id: int
):
    ...

Use * such that the order doesn't matter and all parameters must be passed with key-words

**Comparison**

gt: greater than

le: less than or equal

In [ ]:
@app.get("/items/{item_id}")
async def read_items(
    item_id: Annotated[int, Path(title="The ID of the item to get", ge=1)], q: str
):
    results = {"item_id": item_id}
    if q:
        results.update({"q": q})
    return results